In [2]:
import sys
sys.path.append("../")
import os
import surprise
import papermill as pm
import pandas as pd
from sklearn.metrics import mean_squared_error
from math import sqrt

from reco_utils.common.timer import Timer
from reco_utils.evaluation.python_evaluation import rmse, mae
from reco_utils.recommender.surprise.surprise_utils import predict, compute_ranking_predictions
from surprise.model_selection import train_test_split
from surprise import Dataset
from surprise import Reader

print("System version: {}".format(sys.version))
print("Surprise version: {}".format(surprise.__version__))

System version: 3.7.6 (default, Dec 19 2019, 23:50:13) 
[GCC 7.4.0]
Surprise version: 1.1.0


In [2]:
reader = Reader(sep=',', rating_scale=(1,10))

data = Dataset.load_from_file('../Data/ratings_compressed.csv', reader=reader)

In [3]:
train, test = train_test_split(data, test_size=0.3, random_state=42)

In [5]:
svd = surprise.SVD(random_state=42, n_factors=500, n_epochs=100, verbose=True)

with Timer() as train_time:
    svd.fit(train)

print("Took {} seconds for training.".format(train_time.interval))

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processing epoch 49
Processing

In [6]:
df_test = pd.DataFrame(test, columns=['user_id', 'profile_id', 'rating'])

In [7]:
predictions = predict(svd, df_test, usercol='user_id', itemcol='profile_id')
predictions.head()

,user_id,profile_id,prediction
0,1718,1684,6.782094
1,977,796,7.169265
2,91,1142,9.983137
3,1968,1885,5.608813
4,1452,1812,4.173726


In [8]:
rms = sqrt(mean_squared_error(df_test['rating'].values, predictions['prediction'].values))
rms

2.1050848833350293

In [9]:
test2 = train.build_testset()

df_test = pd.DataFrame(test2, columns=['user_id', 'profile_id', 'rating'])

predictions = predict(svd, df_test, usercol='user_id', itemcol='profile_id')
predictions.head()

,user_id,profile_id,prediction
0,524,1262,6.003640
1,524,773,6.813499
2,524,1933,9.845572
3,524,1250,1.073200
4,524,375,1.273341


In [10]:
rms = sqrt(mean_squared_error(df_test['rating'].values, predictions['prediction'].values))
rms

0.11954922241162601